https://adventofcode.com/2017/day/9

In [57]:
import transitions

In [2]:
with open("data/09.txt") as fh:
    data = fh.read()

In [61]:
class StreamProcessor(transitions.Machine):
    def __init__(self):
        states = ["good", "garbage", "cancel"]
        super().__init__(self, states=states, initial="good")

        self.add_transition("left_curly", "good", "=", after="begin_group")
        self.add_transition("left_curly", "garbage", "=", after="incr_garbage_count")
        self.add_transition("left_curly", "cancel", "garbage")

        self.add_transition("right_curly", "good", "=", after="end_group")
        self.add_transition("right_curly", "garbage", "=", after="incr_garbage_count")
        self.add_transition("right_curly", "cancel", "garbage")

        self.add_transition("lt", "good", "garbage")
        self.add_transition("lt", "garbage", "=", after="incr_garbage_count")
        self.add_transition("lt", "cancel", "garbage")

        self.add_transition("gt", "good", "=")
        self.add_transition("gt", "garbage", "good")
        self.add_transition("gt", "cancel", "garbage")

        self.add_transition("bang", "good", "=")
        self.add_transition("bang", "garbage", "cancel")
        self.add_transition("bang", "cancel", "garbage")

        self.add_transition("other", "good", "=")
        self.add_transition("other", "garbage", "=", after="incr_garbage_count")
        self.add_transition("other", "cancel", "garbage")

        self.group_level = 0
        self.group_score = 0
        self.garbage_count = 0

    def begin_group(self):
        self.group_level += 1

    def end_group(self):
        self.group_score += self.group_level
        self.group_level -= 1
        if self.group_level < 0:
            raise ValueError("Negative group level can't happen")

    def incr_garbage_count(self):
        self.garbage_count += 1

    def step(self, c):
        triggers = {
            "!": "bang",
            "<": "lt",
            ">": "gt",
            "{": "left_curly",
            "}": "right_curly",
        }
        trig = triggers.get(c, "other")
        self.trigger(trig)

    def run(self, data):
        for c in data:
            self.step(c)

In [62]:
sp = StreamProcessor()
sp.run(data)

Part 1

In [63]:
sp.group_score

7640

Part 2

In [64]:
sp.garbage_count

4368